# Example run of MetaSIPSim scripts

Samuel Barnett

### Introduction

Below is a simple example for running MetaSIPSim using 10 random genomes downloaded from NCBI RefSeq. Here I'll take you through:

1. Selecting genomes from a local database
2. Indexing genomes
3. Building an in silico community
4. Selecting incorporators
5. Simulating a density gradient 
6. Generating a configuration file
7. Simulating metagenomic-SIP reads
8. Simulating shotgun metagenomic reads
9. Converting reads from fasta to fastq format

For this example I'll be labeling with 15N and using single heavy-window SIP experimental design. The NCBI RefSeq assemblies in my case is in a directory at `databases/ncbi_genomes/ncbi-genomes-2019-01-25/`.


### Initialization

Import the required python modules, set the working directory and required variables.

In [1]:
import os
import random
import numpy as np
import ConfigParser

## Set the working directory where all files should be stored
workDir = '/home/sam/tmp/MetaSIPSim_example'

## Where are the reference genomes stored?
genomeDir = '/home/sam/databases/ncbi_genomes/ncbi-genomes-2019-01-25/'

## How many processors to use with this simulation
nprocs = 2

In [2]:
## Go into the working directory
%cd $workDir

/home/sam/tmp/MetaSIPSim_example


## 1) Selecting genomes

I've already downloaded the NCBI RefSeq database on January 25th, 2019. I will now take 10 random genomes from this database for our reference set.

In [3]:
## Get all fasta files (.fna) in the database directory
available_genomes = [f for f in os.listdir(genomeDir) if f.endswith('.fna')]

## Select 10 random genomes from the database
ref_set = random.sample(available_genomes, 10)
print(ref_set)

['GCF_002005405.1_ASM200540v1_genomic.fna', 'GCF_001314945.1_ASM131494v1_genomic.fna', 'GCF_002079945.1_ASM207994v1_genomic.fna', 'GCF_000024785.1_ASM2478v1_genomic.fna', 'GCF_002073715.2_ASM207371v2_genomic.fna', 'GCF_000143085.1_ASM14308v1_genomic.fna', 'GCF_000143165.1_ASM14316v1_genomic.fna', 'GCF_000091325.1_ASM9132v1_genomic.fna', 'GCF_000013125.1_ASM1312v1_genomic.fna', 'GCF_900169085.1_SWA-2_genomic.fna']


## 2) Make index file

Now I'll make an index file that will tell the MetaSIPSim scripts what files contain each genome. This will also set the genome IDs, which I will simply use their RefSeq assembly accessions. 

This file should be tab delimited with the first column the genome ID and the second the file name. The index file will be called "genome_index.txt".

In [4]:
with open(os.path.join(workDir, 'genome_index.txt'), 'w') as idxfile:
    for g in ref_set:
        idxfile.write('\t'.join(["_".join(g.split("_", 2)[:2]), g]))
        idxfile.write('\n')
!head genome_index.txt


GCF_002005405.1	GCF_002005405.1_ASM200540v1_genomic.fna
GCF_001314945.1	GCF_001314945.1_ASM131494v1_genomic.fna
GCF_002079945.1	GCF_002079945.1_ASM207994v1_genomic.fna
GCF_000024785.1	GCF_000024785.1_ASM2478v1_genomic.fna
GCF_002073715.2	GCF_002073715.2_ASM207371v2_genomic.fna
GCF_000143085.1	GCF_000143085.1_ASM14308v1_genomic.fna
GCF_000143165.1	GCF_000143165.1_ASM14316v1_genomic.fna
GCF_000091325.1	GCF_000091325.1_ASM9132v1_genomic.fna
GCF_000013125.1	GCF_000013125.1_ASM1312v1_genomic.fna
GCF_900169085.1	GCF_900169085.1_SWA-2_genomic.fna


## 3) Building in silico communities

Now I'll make a community composition table with the abundance of each reference in each sample/library. I'll use two samples for this example. As with many bacterial communities, I'm going to base these two samples on a lognormal species distribution. This will be accomplished with the numpy random lognormal function.

I'll save this community composition file as "community.txt"

In [5]:
with open(os.path.join(workDir, 'community.txt'), 'w') as commfile:
    commfile.write('library\ttaxon_name\trel_abund_perc\trank\n')
    for lib in [1, 2]:
        # Get random abundances for 10 genomes based on lognormal distribution
        abdlist = sorted(list(np.random.lognormal(mean=0, sigma=1, size=10)), reverse=True)
        abdlist = abdlist/sum(abdlist)*100
        # Assign genomes to these abundances
        orderedgenomes = random.sample(ref_set, 10)
        for i in range(0, 10):
            commfile.write('\t'.join([str(lib), 
                                      "_".join(orderedgenomes[i].split("_", 2)[:2]), 
                                      str(abdlist[i]), 
                                      str(i+1)]))
            commfile.write('\n')
                
!cat community.txt


library	taxon_name	rel_abund_perc	rank
1	GCF_001314945.1	32.17446986009239	1
1	GCF_000013125.1	19.053272363328837	2
1	GCF_000143085.1	14.071835106552626	3
1	GCF_002073715.2	9.366510705725819	4
1	GCF_002079945.1	7.405160586239504	5
1	GCF_002005405.1	6.109269108489677	6
1	GCF_000024785.1	4.798331309063747	7
1	GCF_000143165.1	3.7165227052609544	8
1	GCF_000091325.1	2.8901980947625505	9
1	GCF_900169085.1	0.4144301604838983	10
2	GCF_000143085.1	30.55598786114518	1
2	GCF_900169085.1	19.77719280556173	2
2	GCF_000091325.1	12.25134513692992	3
2	GCF_002005405.1	12.027747409371974	4
2	GCF_001314945.1	9.803716840119032	5
2	GCF_002079945.1	6.597278372507359	6
2	GCF_000143165.1	3.8584270764279434	7
2	GCF_000024785.1	1.9537362047275335	8
2	GCF_000013125.1	1.884722681298399	9
2	GCF_002073715.2	1.2898456119109285	10


## 4) Selecting incorporators

I'm just going to randomly select 3 incorporators from each sample to be isotopically labeled. Further, I'll set their mean atom % excess of 15N randomly from a uniform distribution between 50 and 100%. Their standard deviation of isotope labeleing will be 5%.

In [13]:
with open(os.path.join(workDir, 'incorporators.txt'), 'w') as incorpfile:
    incorpfile.write('taxon_name\tlibrary\tpercent_incorporation\tsd_incorporation\n')
    for lib in [1,2]:
        for incorp in random.sample(ref_set, 3):
            incorpfile.write('\t'.join(["_".join(incorp.split("_", 2)[:2]), 
                                        str(lib), str(random.randint(50,100)), '5']))
            incorpfile.write('\n')

!cat incorporators.txt

taxon_name	library	percent_incorporation	sd_incorporation
GCF_002005405.1	1	57	5
GCF_000013125.1	1	58	5
GCF_000143085.1	1	60	5
GCF_002079945.1	2	68	5
GCF_000024785.1	2	63	5
GCF_000143165.1	2	54	5


## 5) Simulating gradient fractions

Now I'll simulate fractions of a CsCl gradient. I'll do this simply by randomly generating fractions with size randomly between 0.003 and 0.010 g/ml starting at 1.670 g/ml and ending at 1.780 g/ml. I'll do this separately for each sample.

The fraction file will be called "fractions.txt".

In [19]:
with open(os.path.join(workDir, 'fractions.txt'), 'w') as fracfile:
    fracfile.write('library\tfraction\tBD_min\tBD_max\tfraction_size\n')
    for lib in [1,2]:
        frac = 1
        BD_max = 1.670
        while BD_max < 1.780:
            frac_size = float(random.randint(3,10))/1000
            BD_min = BD_max
            BD_max = BD_min+frac_size
            fracfile.write('\t'.join([str(lib), str(frac), str(BD_min), str(BD_max), str(frac_size)]))
            fracfile.write('\n')
            frac += 1
            
!cat fractions.txt

library	fraction	BD_min	BD_max	fraction_size
1	1	1.67	1.675	0.005
1	2	1.675	1.684	0.009
1	3	1.684	1.687	0.003
1	4	1.687	1.696	0.009
1	5	1.696	1.7	0.004
1	6	1.7	1.709	0.009
1	7	1.709	1.715	0.006
1	8	1.715	1.724	0.009
1	9	1.724	1.728	0.004
1	10	1.728	1.734	0.006
1	11	1.734	1.738	0.004
1	12	1.738	1.744	0.006
1	13	1.744	1.75	0.006
1	14	1.75	1.76	0.01
1	15	1.76	1.763	0.003
1	16	1.763	1.766	0.003
1	17	1.766	1.771	0.005
1	18	1.771	1.779	0.008
1	19	1.779	1.788	0.009
2	1	1.67	1.676	0.006
2	2	1.676	1.679	0.003
2	3	1.679	1.683	0.004
2	4	1.683	1.693	0.01
2	5	1.693	1.697	0.004
2	6	1.697	1.706	0.009
2	7	1.706	1.712	0.006
2	8	1.712	1.72	0.008
2	9	1.72	1.727	0.007
2	10	1.727	1.732	0.005
2	11	1.732	1.737	0.005
2	12	1.737	1.745	0.008
2	13	1.745	1.749	0.004
2	14	1.749	1.756	0.007
2	15	1.756	1.759	0.003
2	16	1.759	1.766	0.007
2	17	1.766	1.775	0.009
2	18	1.775	1.781	0.006


## 6) Generating configuration file

To run the MetaSIPSim scripts all experimental parameters need to be in a configuration file. We can use the same configuration file for both the metagenomic-SIP and shotgun metagenomic simulations. For more information on each individual parameter, check out the [manual](https://github.com/seb369/MetaSIPSim/blob/master/MetaSIPSim_manual.pdf).

In [22]:
config = ConfigParser.SafeConfigParser()

## Library parameters
config.add_section('Library')
config.set('Library', 'library_list', '1, 2')
config.set('Library', 'window_or_fraction', 'window')
config.set('Library', 'min_bouyant_density_sequenced', '1.72')
config.set('Library', 'max_bouyant_density_sequenced', '1.77')

## Fragment parameters
config.add_section('Fragment')
config.set('Fragment', 'genomeDir', genomeDir)
config.set('Fragment', 'frag_length_distribution', 'skewed-normal,9000,2500,-5')
config.set('Fragment', 'coverage_of_fragments', '10')
config.set('Fragment', 'temp_fragment_file', 'tmp')
config.set('Fragment', 'genome_index_file', 'genome_index.txt')
#config.set('Fragment', 'number_of_iterations', '1') NOT NEEDED IN THIS EXAMPLE

## Gradient parameters
config.add_section('Gradient')
config.set('Gradient', 'temperature', '293.15')
config.set('Gradient', 'avg_density', '1.69')
config.set('Gradient', 'angular_velocity', '33172837')
config.set('Gradient', 'min_rotation_radius', '2.6')
config.set('Gradient', 'max_rotation_radius', '4.85')
config.set('Gradient', 'tube_angle', '28.6')
config.set('Gradient', 'tube_radius', '0.66')
config.set('Gradient', 'tube_height', '4.7')
config.set('Gradient', 'fraction_frag_in_DBL', '0.001')
config.set('Gradient', 'isotope', 'N')
#config.set('Gradient', 'isotope', 'C') # ANOTHER POSSIBLE OPTION

## Model parameters
config.add_section('Model')
config.set('Model', 'min_bouyant_density', '1.67')
config.set('Model', 'max_bouyant_density', '1.78')
config.set('Model', 'bouyant_density_step', '0.0001')
config.set('Model', 'fraction_table_file', 'fractions.txt')

## Community parameters
config.add_section('Community')
config.set('Community', 'community_file', 'community.txt')
config.set('Community', 'incorporator_file', 'incorporators.txt')

## Sequencing parameters
config.add_section('Sequencing')
config.set('Sequencing', 'max_read_length', '151')
config.set('Sequencing', 'avg_insert_size', '1000')
config.set('Sequencing', 'stddev_insert_size', '5')
config.set('Sequencing', 'final_number_of_sequences', '100000')
config.set('Sequencing', 'number_of_sequences_per_iteration', '100000')

## Other parameters
config.add_section('Other')
config.set('Other', 'temp_directory', 'tmp')
config.set('Other', 'threads', '2')
config.set('Other', 'logfile', 'example_simulation.log')
config.set('Other', 'endpoint', 'read_sequences')
#config.set('Other', 'endpoint', 'fragment_list') # ANOTHER POSSIBLE OPTION
#config.set('Other', 'endpoint', 'read_list') # ANOTHER POSSIBLE OPTION

# Writing our configuration file to 'example.cfg'
with open(os.path.join(workDir, 'example_parameters.cfg'), 'wb') as configfile:
    config.write(configfile)

## 7) Simulating metagenomic-SIP reads

Now I'll run the function `SIPSim_metagenome.py` to simulate reads from a metagenomic-SIP experiment with our in silico community and incorporators.

In [23]:
!python /home/sam/notebooks/MetaSIPSim/bin/SIPSim_metagenome.py example_parameters.cfg

Running SIPSim_metagenome
This program was writen by Samuel Barnett (seb369@cornell.edu)

This run was started on 06/08/19 at 15:24:47


You have chosen to get sequences of simulated, sequenced SIP metagenome reads.
You have selected to simulate metagenome for a SIP gradient between buoyant densities: 1.72 and 1.77.

Your community abundance file is: community.txt

Your incorporator assignment file is: incorporators.txt

You are simulating with the isotope of N

It took 0.963 seconds to get these models.
It took 0.937 seconds to get these models.

Building fragments

Temporary fragment directory already exists. Your current files may be overwriten.


It took 7.35 seconds to build the fragments

----------

Starting library 1

Starting library 1 in window BD:1.72-1.77
Writing fragments to file
It took 7.932 seconds to write fragment file.
It took 1.161 seconds to generate reads. Now building map.
Writing read sequences
It took 24.654 seconds to make this fasta
It took 25.656 seconds to 

## 8) Simulating shotgun metagenomic reads

Now I'll run the function `nonSIP_metagenome.py` to simulate reads from a conventional shotgun metagenome experiment with our in silico community.

In [24]:
!python /home/sam/notebooks/MetaSIPSim/bin/nonSIP_metagenome.py example_parameters.cfg

Running nonSIP_metagenome
This program was writen by Samuel Barnett (seb369@cornell.edu)

This run was started on 06/08/19 at 15:27:44


You have chosen to get sequences of simulated, sequenced SIP metagenome reads.

Building fragments

Temporary fragment directory already exists. Your current files may be overwriten.


It took 7.072 seconds to build the fragments

----------

Starting library 1

Writing fragments to file
It took 0.4 seconds to write fragment file.
It took 2.33 seconds to generate reads. Now building map.
Writing read sequences
It took 20.154 seconds to make this fasta
It took 22.306 seconds to run library 1 iteration 1

It took 22.884 seconds to run the whole library 1

----------

Starting library 2

Writing fragments to file
It took 0.402 seconds to write fragment file.
It took 2.04 seconds to generate reads. Now building map.
Writing read sequences
It took 17.05 seconds to make this fasta
It took 18.946 seconds to run library 2 iteration 1

It took 19.493 seconds t

## 9) Converting reads from fasta to fastq format

The output from the previous two sections is fasta formated reads. For most applications, you want fastq formated reads. Here I'll use the fasta2fastq.py script to convert the reads.

First I need to get a list of the fasta files that were just generated (those ending in "fasta.gz")

In [25]:
fasta_files = [f for f in os.listdir(workDir) if f.endswith('.fasta.gz')]

Now I'll run the fasta2fastq.py script separately for each fasta file. This script needs to know which direction the reads are (forward or reverse). Those with "reads_f" are forward and with "reads_r" are reverse.  I'll use the NovaSeq error model for this run. I have provided the error models from InSilicoSeq along with this toolkit.

Note: this might take a little while to run fully.

In [29]:
for fasta in sorted(fasta_files):
    print("Running " + fasta)
    # Uncompress the fasta file
    cmd = ' '.join(['pigz -d -k -p', str(nprocs), fasta])
    print(cmd)
    os.system(cmd)
    # Convert to fastq file
    fastafile = fasta.replace(".gz", "")
    # Which direction are these reads?
    if fastafile.endswith('reads_f.fasta'):
        direction = 'forward'
    elif fastafile.endswith('reads_r.fasta'):
        direction = 'reverse'
    # Make command arguments (i d e l t p)
    args = ' '. join([fastafile, 
                      direction, 
                      '/home/sam/notebooks/MetaSIPSim/ISS_error_models/NovaSeq', 
                      '151', 
                      'tmp', 
                      '2'])
    # Run fasta2fastq.py
    cmd = ' '.join(['python /home/sam/notebooks/MetaSIPSim/bin/fasta2fastq.py', 
                    args])
    print(cmd)
    os.system(cmd)
    # Remove uncompressed fasta file
    cmd = ' '.join(['rm', fastafile])
    print(cmd)
    os.system(cmd)
    print('\n')

Running library_1_window_1.72-1.77_reads_f.fasta.gz
pigz -d -k -p 2 library_1_window_1.72-1.77_reads_f.fasta.gz
python /home/sam/notebooks/MetaSIPSim/bin/fasta2fastq.py library_1_window_1.72-1.77_reads_f.fasta forward /home/sam/notebooks/MetaSIPSim/ISS_error_models/NovaSeq 151 tmp 2
rm library_1_window_1.72-1.77_reads_f.fasta


Running library_1_window_1.72-1.77_reads_r.fasta.gz
pigz -d -k -p 2 library_1_window_1.72-1.77_reads_r.fasta.gz
python /home/sam/notebooks/MetaSIPSim/bin/fasta2fastq.py library_1_window_1.72-1.77_reads_r.fasta reverse /home/sam/notebooks/MetaSIPSim/ISS_error_models/NovaSeq 151 tmp 2
rm library_1_window_1.72-1.77_reads_r.fasta


Running library_2_window_1.72-1.77_reads_f.fasta.gz
pigz -d -k -p 2 library_2_window_1.72-1.77_reads_f.fasta.gz
python /home/sam/notebooks/MetaSIPSim/bin/fasta2fastq.py library_2_window_1.72-1.77_reads_f.fasta forward /home/sam/notebooks/MetaSIPSim/ISS_error_models/NovaSeq 151 tmp 2
rm library_2_window_1.72-1.77_reads_f.fasta


Running li

## List files generated through this tutorial

Check out all the files generated in this tutorial!

In [30]:
!ls -lh $workDir

total 90M
-rw-rw-r-- 1 sam sam   20 Aug  6 15:25 BD_min_max_list.pkl
-rw-rw-r-- 1 sam sam  810 Aug  6 14:55 community.txt
-rw-rw-r-- 1 sam sam 1.1K Aug  6 15:23 example_parameters.cfg
-rw-rw-r-- 1 sam sam 1.1K Aug  6 15:28 example_simulation.log
-rw-rw-r-- 1 sam sam  864 Aug  6 15:09 fractions.txt
-rw-rw-r-- 1 sam sam  546 Aug  6 14:55 genome_index.txt
-rw-rw-r-- 1 sam sam  196 Aug  6 14:57 incorporators.txt
-rw-rw-r-- 1 sam sam 895K Aug  6 15:25 library_1_window_1.72-1.77_fragments.txt.gz
-rw-rw-r-- 1 sam sam 4.6M Aug  6 15:25 library_1_window_1.72-1.77_reads_f.fasta.gz
-rw-rw-r-- 1 sam sam 5.8M Aug  6 15:48 library_1_window_1.72-1.77_reads_f.fastq.gz
-rw-rw-r-- 1 sam sam 4.6M Aug  6 15:25 library_1_window_1.72-1.77_reads_r.fasta.gz
-rw-rw-r-- 1 sam sam 6.4M Aug  6 15:49 library_1_window_1.72-1.77_reads_r.fastq.gz
-rw-rw-r-- 1 sam sam 913K Aug  6 15:25 library_2_window_1.72-1.77_fragments.txt.gz
-rw-rw-r-- 1 sam sam 4.6M Aug  6 15:26 library_2_window_1.72-1.77_reads_f.fasta.gz
-rw-rw-

## The End

Thank you for checking out this tutorial and for considering MetaSIPSim for your simulation needs!